# Análise de Desmatamento no Brasil

Este notebook apresenta uma análise exploratória dos dados de desmatamento do PRODES/INPE, com foco especial no estado do Piauí e no bioma Cerrado.

## Objetivos
1. Analisar a evolução temporal do desmatamento
2. Comparar desmatamento entre biomas e estados
3. Identificar tendências e padrões
4. Gerar insights sobre a situação do Piauí

## Fonte de Dados
- **PRODES/INPE**: Programa de Monitoramento do Desmatamento da Floresta Amazônica Brasileira por Satélite
- **TerraBrasilis**: http://terrabrasilis.dpi.inpe.br/

## 1. Configuração do Ambiente

In [ ]:
# Imports
import sys
from pathlib import Path

# Adicionar diretório raiz ao path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configurações
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 12
pd.set_option('display.max_columns', None)

# Cores para biomas
CORES_BIOMAS = {
    'Amazônia': '#228B22',
    'Cerrado': '#DAA520',
    'Mata Atlântica': '#2E8B57',
    'Caatinga': '#DEB887',
    'Pampa': '#90EE90',
    'Pantanal': '#4682B4'
}

print("Ambiente configurado com sucesso!")

In [ ]:
# Importar módulos do projeto
from src.utils.data_loader import DataLoader
from src.utils.data_processor import DataProcessor
from src.utils.config import ESTADOS_INFO, BIOMAS, DADOS_RECENTES_CERRADO

# Inicializar componentes
loader = DataLoader()
processor = DataProcessor()

print("Módulos do projeto carregados!")

## 2. Carregamento dos Dados

In [ ]:
# Carregar dados de desmatamento
df = loader.load_data()

print(f"Total de registros: {len(df):,}")
print(f"\nColunas: {df.columns.tolist()}")
print(f"\nPeríodo: {df['ano'].min()} a {df['ano'].max()}")
print(f"\nBiomas: {df['bioma'].unique().tolist()}")
print(f"\nEstados: {df['estado'].nunique()}")

In [ ]:
# Visualizar primeiros registros
df.head(10)

In [ ]:
# Estatísticas descritivas
df.describe()

## 3. Análise Temporal do Desmatamento

In [ ]:
# Desmatamento total por ano
desmatamento_anual = df.groupby('ano')['area_km2'].sum().reset_index()
desmatamento_anual.columns = ['ano', 'total_km2']

# Gráfico de evolução temporal
fig = px.area(
    desmatamento_anual,
    x='ano',
    y='total_km2',
    title='Evolução do Desmatamento no Brasil (2000-2025)',
    labels={'ano': 'Ano', 'total_km2': 'Área Desmatada (km²)'},
    color_discrete_sequence=['#FF6B6B']
)
fig.update_layout(
    hovermode='x unified',
    yaxis_tickformat=','
)
fig.show()

In [ ]:
# Desmatamento por bioma ao longo do tempo
desmat_bioma_ano = df.groupby(['ano', 'bioma'])['area_km2'].sum().reset_index()

fig = px.line(
    desmat_bioma_ano,
    x='ano',
    y='area_km2',
    color='bioma',
    title='Desmatamento por Bioma (2000-2025)',
    labels={'ano': 'Ano', 'area_km2': 'Área Desmatada (km²)', 'bioma': 'Bioma'},
    color_discrete_map=CORES_BIOMAS
)
fig.update_traces(line=dict(width=2))
fig.update_layout(hovermode='x unified')
fig.show()

In [ ]:
# Calcular variação anual
desmatamento_anual['variacao_pct'] = desmatamento_anual['total_km2'].pct_change() * 100

# Identificar anos com maior aumento e redução
print("Anos com MAIOR AUMENTO no desmatamento:")
print(desmatamento_anual.nlargest(5, 'variacao_pct')[['ano', 'total_km2', 'variacao_pct']])

print("\nAnos com MAIOR REDUÇÃO no desmatamento:")
print(desmatamento_anual.nsmallest(5, 'variacao_pct')[['ano', 'total_km2', 'variacao_pct']])

## 4. Análise por Estado

In [ ]:
# Top 10 estados com maior desmatamento acumulado
desmat_estado = df.groupby('estado')['area_km2'].sum().reset_index()
desmat_estado = desmat_estado.sort_values('area_km2', ascending=False)

top10 = desmat_estado.head(10)

fig = px.bar(
    top10,
    x='estado',
    y='area_km2',
    title='Top 10 Estados com Maior Desmatamento Acumulado (2000-2025)',
    labels={'estado': 'Estado', 'area_km2': 'Área Desmatada (km²)'},
    color='area_km2',
    color_continuous_scale='Reds',
    text='area_km2'
)
fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Heatmap: Desmatamento por Estado e Ano
pivot = df.pivot_table(
    values='area_km2',
    index='estado',
    columns='ano',
    aggfunc='sum',
    fill_value=0
)

# Ordenar por total
pivot = pivot.loc[pivot.sum(axis=1).sort_values(ascending=False).head(15).index]

fig = px.imshow(
    pivot,
    title='Heatmap: Desmatamento por Estado e Ano (Top 15)',
    labels={'x': 'Ano', 'y': 'Estado', 'color': 'Área (km²)'},
    color_continuous_scale='YlOrRd',
    aspect='auto'
)
fig.update_layout(height=600)
fig.show()

## 5. Foco: Piauí

In [ ]:
# Filtrar dados do Piauí
df_piaui = df[df['estado'] == 'PI'].copy()

print(f"Registros do Piauí: {len(df_piaui)}")
print(f"Biomas no Piauí: {df_piaui['bioma'].unique().tolist()}")
print(f"Total desmatado: {df_piaui['area_km2'].sum():,.2f} km²")

In [ ]:
# Evolução do desmatamento no Piauí
piaui_anual = df_piaui.groupby('ano')['area_km2'].sum().reset_index()

fig = px.bar(
    piaui_anual,
    x='ano',
    y='area_km2',
    title='Desmatamento no Piauí por Ano',
    labels={'ano': 'Ano', 'area_km2': 'Área Desmatada (km²)'},
    color='area_km2',
    color_continuous_scale='Oranges'
)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Comparar Piauí com estados vizinhos (Cerrado)
estados_cerrado = ['MA', 'PI', 'BA', 'TO', 'GO', 'MT', 'MS', 'MG']

df_cerrado = df[df['estado'].isin(estados_cerrado)].copy()
cerrado_estado = df_cerrado.groupby('estado')['area_km2'].sum().reset_index()
cerrado_estado = cerrado_estado.sort_values('area_km2', ascending=True)

# Destacar Piauí
cores = ['lightgray' if e != 'PI' else '#FF6B6B' for e in cerrado_estado['estado']]

fig = px.bar(
    cerrado_estado,
    x='area_km2',
    y='estado',
    orientation='h',
    title='Desmatamento Acumulado: Piauí vs Estados do Cerrado',
    labels={'area_km2': 'Área Desmatada (km²)', 'estado': 'Estado'},
    color='estado',
    color_discrete_sequence=cores
)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Dados recentes do Cerrado 2025 (INPE)
print("=" * 60)
print("DADOS OFICIAIS PRODES/INPE - CERRADO 2025 (Preliminares)")
print("=" * 60)

df_cerrado_2025 = pd.DataFrame(DADOS_RECENTES_CERRADO)
df_cerrado_2025 = df_cerrado_2025.sort_values('area_km2', ascending=False)

print(f"\nTotal desmatado no Cerrado (ago/2024-jul/2025): {df_cerrado_2025['area_km2'].sum():,.0f} km²")
print(f"\nRanking dos estados:")
for i, row in df_cerrado_2025.iterrows():
    rank = df_cerrado_2025.index.get_loc(i) + 1
    destaque = " <<<" if row['estado'] == 'PI' else ""
    print(f"{rank}º {row['estado']}: {row['area_km2']:,.0f} km²{destaque}")

## 6. Análise de Tendências

In [ ]:
# Calcular tendência com regressão linear
from scipy import stats

# Tendência geral
slope, intercept, r_value, p_value, std_err = stats.linregress(
    desmatamento_anual['ano'],
    desmatamento_anual['total_km2']
)

# Criar linha de tendência
desmatamento_anual['tendencia'] = intercept + slope * desmatamento_anual['ano']

fig = go.Figure()

# Dados reais
fig.add_trace(go.Scatter(
    x=desmatamento_anual['ano'],
    y=desmatamento_anual['total_km2'],
    mode='lines+markers',
    name='Desmatamento Real',
    line=dict(color='#FF6B6B', width=2)
))

# Linha de tendência
fig.add_trace(go.Scatter(
    x=desmatamento_anual['ano'],
    y=desmatamento_anual['tendencia'],
    mode='lines',
    name=f'Tendência (R²={r_value**2:.3f})',
    line=dict(color='blue', width=2, dash='dash')
))

fig.update_layout(
    title='Tendência do Desmatamento no Brasil',
    xaxis_title='Ano',
    yaxis_title='Área Desmatada (km²)',
    hovermode='x unified'
)

fig.show()

tendencia = "aumento" if slope > 0 else "redução"
print(f"\nTendência: {tendencia} de {abs(slope):,.1f} km²/ano")
print(f"R² = {r_value**2:.4f} (qualidade do ajuste)")

## 7. Distribuição por Bioma

In [ ]:
# Desmatamento total por bioma
desmat_bioma = df.groupby('bioma')['area_km2'].sum().reset_index()
desmat_bioma = desmat_bioma.sort_values('area_km2', ascending=False)

# Gráfico de pizza
fig = px.pie(
    desmat_bioma,
    values='area_km2',
    names='bioma',
    title='Distribuição do Desmatamento por Bioma (2000-2025)',
    color='bioma',
    color_discrete_map=CORES_BIOMAS,
    hole=0.4
)
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

In [ ]:
# Comparação: Amazônia vs Cerrado (últimos 10 anos)
df_recente = df[df['ano'] >= 2015]

amaz_cerr = df_recente[df_recente['bioma'].isin(['Amazônia', 'Cerrado'])]
amaz_cerr_ano = amaz_cerr.groupby(['ano', 'bioma'])['area_km2'].sum().reset_index()

fig = px.bar(
    amaz_cerr_ano,
    x='ano',
    y='area_km2',
    color='bioma',
    barmode='group',
    title='Amazônia vs Cerrado: Desmatamento Anual (2015-2025)',
    labels={'ano': 'Ano', 'area_km2': 'Área Desmatada (km²)', 'bioma': 'Bioma'},
    color_discrete_map=CORES_BIOMAS
)
fig.show()

## 8. Resumo e Conclusões

In [ ]:
# Estatísticas resumidas
print("=" * 70)
print("RESUMO DA ANÁLISE DE DESMATAMENTO")
print("=" * 70)

total_desmatado = df['area_km2'].sum()
media_anual = df.groupby('ano')['area_km2'].sum().mean()

print(f"\n📊 ESTATÍSTICAS GERAIS (2000-2025):")
print(f"   • Total desmatado: {total_desmatado:,.0f} km²")
print(f"   • Média anual: {media_anual:,.0f} km²")
print(f"   • Número de estados afetados: {df['estado'].nunique()}")

# Top 3 biomas
print(f"\n🌳 TOP 3 BIOMAS MAIS AFETADOS:")
for i, row in desmat_bioma.head(3).iterrows():
    pct = (row['area_km2'] / total_desmatado) * 100
    print(f"   {desmat_bioma.index.get_loc(i)+1}. {row['bioma']}: {row['area_km2']:,.0f} km² ({pct:.1f}%)")

# Top 3 estados
print(f"\n📍 TOP 3 ESTADOS MAIS AFETADOS:")
for i, row in desmat_estado.head(3).iterrows():
    pct = (row['area_km2'] / total_desmatado) * 100
    print(f"   {desmat_estado.index.get_loc(i)+1}. {row['estado']}: {row['area_km2']:,.0f} km² ({pct:.1f}%)")

# Piauí
piaui_total = df_piaui['area_km2'].sum()
piaui_ranking = desmat_estado[desmat_estado['estado'] == 'PI'].index[0] + 1
print(f"\n🎯 PIAUÍ:")
print(f"   • Ranking nacional: {piaui_ranking}º lugar")
print(f"   • Total desmatado: {piaui_total:,.0f} km²")
print(f"   • Participação nacional: {(piaui_total/total_desmatado)*100:.2f}%")

print("\n" + "=" * 70)

## 9. Exportação dos Dados

In [ ]:
# Exportar dados processados
output_dir = project_root / 'data' / 'processed'
output_dir.mkdir(parents=True, exist_ok=True)

# Dados completos
df.to_csv(output_dir / 'desmatamento_completo.csv', index=False, encoding='utf-8-sig')

# Dados do Piauí
df_piaui.to_csv(output_dir / 'desmatamento_piaui.csv', index=False, encoding='utf-8-sig')

# Resumo anual
desmatamento_anual.to_csv(output_dir / 'desmatamento_anual.csv', index=False, encoding='utf-8-sig')

# Resumo por estado
desmat_estado.to_csv(output_dir / 'desmatamento_por_estado.csv', index=False, encoding='utf-8-sig')

print(f"Dados exportados para: {output_dir}")
for f in output_dir.glob('*.csv'):
    print(f"  • {f.name}")

---

## Conclusões Principais

1. **Amazônia e Cerrado** concentram a maior parte do desmatamento brasileiro
2. O **Cerrado** tem apresentado índices preocupantes, especialmente nos estados do MATOPIBA
3. O **Piauí** está entre os principais contribuintes para o desmatamento do Cerrado
4. Os dados de 2025 mostram redução em comparação a 2024, mas o nível ainda é elevado
5. É necessário fortalecer políticas de preservação, especialmente no Cerrado

---

*Análise realizada com dados do PRODES/INPE*